In [1]:
!pip install protobuf==3.20.3 --force-reinstall


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tens

In [2]:
# if you need (uncomment if missing in your environment)
#!pip -q install transformers datasets scikit-learn torch sentencepiece

import re
import numpy as np
import pandas as pd
from dataclasses import dataclass

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

import torch
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)


2026-01-10 12:13:09.699370: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768047189.892789      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768047189.949890      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
!git clone https://github.com/hadyelsahar/large-arabic-sentiment-analysis-resouces.git 

Cloning into 'large-arabic-sentiment-analysis-resouces'...
remote: Enumerating objects: 60, done.
remote: Total 60 (delta 0), reused 0 (delta 0), pack-reused 60 (from 1)
Receiving objects: 100% (60/60), 19.14 MiB | 22.94 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [4]:
!git clone https://github.com/mohataher/arabic-stop-words.git

Cloning into 'arabic-stop-words'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 48 (delta 3), reused 2 (delta 0), pack-reused 37 (from 1)
Receiving objects: 100% (48/48), 25.35 KiB | 12.67 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [5]:
!pip install pyarabic

import pyarabic.araby as araby
import pyarabic.number as number

In [6]:
AAT = pd.read_csv('large-arabic-sentiment-analysis-resouces/datasets/ATT.csv')
HTL = pd.read_csv('large-arabic-sentiment-analysis-resouces/datasets/HTL.csv')
MOV = pd.read_csv('large-arabic-sentiment-analysis-resouces/datasets/MOV.csv')
PROD = pd.read_csv('large-arabic-sentiment-analysis-resouces/datasets/PROD.csv')
RES = pd.read_csv('large-arabic-sentiment-analysis-resouces/datasets/RES.csv')
RES1 = pd.read_csv('large-arabic-sentiment-analysis-resouces/datasets/RES1.csv')
RES2 = pd.read_csv('large-arabic-sentiment-analysis-resouces/datasets/RES2.csv')

In [7]:
!ls arabic-stop-words


fahrasa.png  LICENSE  list.txt	README.md


In [8]:
stopwords = pd.read_csv("arabic-stop-words/list.txt", header=None, names=["word"])
stopwords.head()

,word
0,،
1,ـ
2,ء
3,ءَ
4,آ


In [9]:
# --- 1) Put all domain dataframes here in one df
DOMAIN_DFS = {
    "AAT": AAT,
    "HTL": HTL,
    "MOV": MOV,
    "PROD": PROD,
    "RES": RES,
    "RES1": RES1,
    "RES2": RES2,
}

# --- 2) force the names if your columns are non-standard ---
TEXT_COL = None   # e.g., "text" / "review" / "sentence" / "body" ...
LABEL_COL = None  # e.g., "label" / "sentiment" / "polarity" ...

COMMON_TEXT_COLS = ["text", "Text", "sentence", "Sentence", "review", "Review", "body", "Body"]
COMMON_LABEL_COLS = ["label", "Label", "sentiment", "Sentiment", "polarity", "Polarity"]

def _guess_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def unify_one(df, domain):
    global TEXT_COL, LABEL_COL
    if TEXT_COL is None:
        TEXT_COL = _guess_col(df, COMMON_TEXT_COLS)
    if LABEL_COL is None:
        LABEL_COL = _guess_col(df, COMMON_LABEL_COLS)

    if TEXT_COL is None or LABEL_COL is None:
        raise ValueError(
            f"Could not auto-detect text/label columns for domain {domain}. "
            f"Set TEXT_COL and LABEL_COL manually above."
        )

    out = df[[TEXT_COL, LABEL_COL]].copy()
    out.columns = ["text", "label"]
    out["domain"] = domain
    return out

all_parts = []
for dname, dframe in DOMAIN_DFS.items():
    all_parts.append(unify_one(dframe, dname))

data = pd.concat(all_parts, ignore_index=True)
data = data.dropna(subset=["text", "label"]).reset_index(drop=True)

data.head(), data.shape


(                                                text  label domain
 0  حمام الكبريت \r\nنصحوني بتجربة حمام الكبريت. ي...    1.0    AAT
 1  قلعة ساحرة \r\nمنظر خلاب للمدينة من أعلى القلع...    1.0    AAT
 2  تبليسي \r\nتبليسي جورجيا\r\nمن أجمل المدن التي...    1.0    AAT
 3  جولة على المدينة القديمة تبليسي < شاردن \r\nمم...    1.0    AAT
 4  احلي اجازه لمحبي الطبيعه \r\nالمناظر الخلابه ا...    1.0    AAT,
 (45498, 3))

In [10]:
# Make a flat set of stopwords from your uploaded dataframe (any column will work)
stopword_set = set()

for col in stopwords.columns:
    vals = stopwords[col].astype(str).str.strip()
    stopword_set.update([v for v in vals if v and v != "nan"])

len(stopword_set), list(next(iter(stopword_set)) for _ in range(min(10, len(stopword_set))))


(799,
 ['اللواتي',
  'اللواتي',
  'اللواتي',
  'اللواتي',
  'اللواتي',
  'اللواتي',
  'اللواتي',
  'اللواتي',
  'اللواتي',
  'اللواتي'])

In [11]:
# --- Normalization helpers (simple + practical) ---
def normalize_arabic(text: str) -> str:
    text = str(text)

    # remove tatweel (kashida) and normalize common hamza/aleph variants
    text = re.sub(r"[\u0640]", "", text)  # tatweel
    text = araby.normalize_hamza(text)     # normalize hamza forms (PyArabic)
    text = araby.strip_diacritics(text)    # remove diacritics (tashkeel)

    # basic spacing cleanup
    text = re.sub(r"\s+", " ", text).strip()
    return text

def simple_tokenize(text: str) -> list:
    # PyArabic has a tokenizer that returns word tokens (good baseline)
    try:
        tokens = list(araby.tokenize(text))
    except Exception:
        # fallback: whitespace split (safe)
        tokens = text.split()
    return [t for t in tokens if t]

def remove_stopwords(tokens: list, stop_set: set) -> list:
    return [t for t in tokens if t not in stop_set and t.strip() != ""]

def preprocess(text: str) -> str:
    norm = normalize_arabic(text)
    tokens = simple_tokenize(norm)
    tokens = remove_stopwords(tokens, stopword_set)
    return " ".join(tokens)


In [12]:
# mapping the labels

label_map = {
    "positive": "positive",
    "pos": "positive",
    "neg": "negative",
    "negative": "negative",
    "neutral": "neutral",
    "neu": "neutral",
    "0": "neutral",
    "1": "positive",
    "-1": "negative",
    0: "neutral",
    1: "positive",
    -1: "negative",
}

data["text_clean"] = data["text"].astype(str).apply(preprocess)
data["label_std"] = data["label"].apply(lambda x: label_map.get(x, str(x).lower()))

data[["text", "text_clean", "label", "label_std", "domain"]].head()


,text,text_clean,label,label_std,domain
0,حمام الكبريت \r\nنصحوني بتجربة حمام الكبريت. ي...,حمام الكبريت نصحوني بتجربة حمام الكبريت . يمكن...,1.0,positive,AAT
1,قلعة ساحرة \r\nمنظر خلاب للمدينة من أعلى القلع...,قلعة ساحرة منظر خلاب للمدينة ءعلى القلعة . يوج...,1.0,positive,AAT
2,تبليسي \r\nتبليسي جورجيا\r\nمن أجمل المدن التي...,تبليسي تبليسي جورجيا ءجمل المدن زرتها حياتي شع...,1.0,positive,AAT
3,جولة على المدينة القديمة تبليسي < شاردن \r\nمم...,جولة المدينة القديمة تبليسي < شاردن ممتعة الجو...,1.0,positive,AAT
4,احلي اجازه لمحبي الطبيعه \r\nالمناظر الخلابه ا...,احلي اجازه لمحبي الطبيعه المناظر الخلابه الطبي...,1.0,positive,AAT


In [13]:
train_df, temp_df = train_test_split(
    data, test_size=0.20, stratify=data["label_std"], random_state=42
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.50, stratify=temp_df["label_std"], random_state=42
)

len(train_df), len(val_df), len(test_df), train_df["label_std"].value_counts()


(36398,
 4550,
 4550,
 label_std
 positive    26402
 negative     7469
 neutral      2527
 Name: count, dtype: int64)

In [14]:
training_args = TrainingArguments(
    output_dir="outputs/arabic_sentiment",
    eval_strategy="epoch",        
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    
    # Core training parameters
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    report_to="none",
)


In [15]:
# Recommended compact Arabic encoder for classification (BERT-style):
# asafaya/bert-base-arabic is a strong, widely used Arabic baseline model. :contentReference[oaicite:2]{index=2}
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "asafaya/bert-base-arabic"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3
)

label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {v: k for k, v in label2id.items()}

def encode_batch(df: pd.DataFrame):
    return tokenizer(
        df["text_clean"].tolist(),
        padding=True,
        truncation=True,
        max_length=256
    )

class PandasDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.enc = encode_batch(df)
        self.labels = [label2id[l] for l in df["label_std"].tolist()]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.enc.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_ds = PandasDataset(train_df)
val_ds   = PandasDataset(val_df)
test_ds  = PandasDataset(test_df)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {
        "accuracy": float(accuracy_score(labels, preds)),
        "f1_macro": float(f1_score(labels, preds, average="macro"))
    }



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
)


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at asafaya/bert-base-arabic and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
trainer.train()

# Evaluate on test
test_preds = trainer.predict(test_ds)
y_true = test_preds.label_ids
y_pred = np.argmax(test_preds.predictions, axis=1)

print("Test accuracy:", accuracy_score(y_true, y_pred))
print("Test macro-F1:", f1_score(y_true, y_pred, average="macro"))
print("\nClassification report (test):")
print(classification_report(y_true, y_pred, target_names=[id2label[i] for i in range(3)]))


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.368700,0.308164,0.889011,0.740654
2,0.258600,0.309657,0.894945,0.749052
3,0.115800,0.387100,0.904396,0.772609


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Test accuracy: 0.8982417582417582
Test macro-F1: 0.751030540078813

Classification report (test):
              precision    recall  f1-score   support

    negative       0.87      0.86      0.86       934
     neutral       0.48      0.41      0.44       316
    positive       0.94      0.96      0.95      3300

    accuracy                           0.90      4550
   macro avg       0.76      0.74      0.75      4550
weighted avg       0.89      0.90      0.90      4550



In [17]:
save_dir = "models/arabic_sentiment_model"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

print("Saved model + tokenizer to:", save_dir)


Saved model + tokenizer to: models/arabic_sentiment_model


In [18]:
import torch
import numpy as np

def predict_with_key_tokens(text, top_k=5, max_words=40):
    model.eval()
    device = model.device

    # simple whitespace split
    words = text.split()
    if len(words) > max_words:
        words = words[:max_words]

    # 1) baseline full prediction
    enc_full = tokenizer(" ".join(words), return_tensors="pt", truncation=True, max_length=256)
    enc_full = {k: v.to(device) for k, v in enc_full.items()}   # <<< FIX
    with torch.no_grad():
        logits_full = model(**enc_full).logits

    probs_full = torch.softmax(logits_full, dim=-1).cpu().numpy()[0]
    pred_id = int(np.argmax(probs_full))

    # 2) leave-one-out token importance
    token_scores = []

    for i, w in enumerate(words):
        reduced = words[:i] + words[i+1:]
        if not reduced:  # skip single-token edge
            continue

        enc = tokenizer(" ".join(reduced), return_tensors="pt", truncation=True, max_length=256)
        enc = {k: v.to(device) for k, v in enc.items()}        # <<< FIX
        with torch.no_grad():
            logits = model(**enc).logits

        probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]
        drop = probs_full[pred_id] - probs[pred_id]
        token_scores.append((w, float(drop)))

    token_scores = sorted(token_scores, key=lambda x: x[1], reverse=True)
    key_tokens = [w for w, s in token_scores[:top_k]]

    label = ["negative", "neutral", "positive"][pred_id]

    return {
        "label": label,
        "probabilities": probs_full.tolist(),
        "key_words": key_tokens
    }


In [19]:
# for 1 example/dummyexample
example = "هذا الفيلم كان ممتعًا جدًا لكن النهاية كانت ضعيفة."
predict_with_key_tokens(example, top_k=5)


{'label': 'positive',
 'probabilities': [0.2891921401023865, 0.19323374330997467, 0.5175741314888],
 'key_words': ['ممتعًا', 'لكن', 'كانت', 'ضعيفة.', 'هذا']}

In [20]:
test_df


,text,label,domain,text_clean,label_std
7607,لا انصح \r\nلا انصح بهذا الفندق للعائلات من من...,-1.0,HTL,انصح انصح الفندق للعاءلات منطقة البحر المتوسط ...,negative
42820,الذ واطعم مطبق ذقته في حياتي,1.0,RES1,الذ واطعم مطبق ذقته حياتي,positive
34907,كان الجو حلو ...جدا في الرياض \n \n وكنت موا...,1.0,RES1,الجو حلو ... الرياض وكنت مواعد العاءلة بطلعه ت...,positive
36459,يتميز المطعم بوجود وجبات على المنيو او حسب الر...,1.0,RES1,يتميز المطعم بوجود وجبات المنيو الرغبة بالاختي...,positive
23775,جيد وليس أكثر \nوجدت المطعم مرشح من قبل العديد...,0.0,RES,جيد ءكثر وجدت المطعم مرشح العديد تقارير مواقع ...,neutral
...,...,...,...,...,...
23616,ﺣﻜﺎﻳﺔ \nعلي طول في شارع صغير جنب البانتيون بال...,1.0,RES,ﺣﻜﺎﻳﺔ طول شارع صغير جنب البانتيون بالصدفة دخلت...,positive
17755,ذاكرة الجسد أضحى جسداً دون ذاكرة تذكر.. ذاكرة ...,-1.0,MOV,ذاكرة الجسد ءضحى جسدا ذاكرة تذكر .. ذاكرة الجس...,negative
36663,جربت عنده الأضلاع 3 مرات وكان ممتازة جداً أعجب...,1.0,RES1,جربت عنده الءضلاع 3 مرات ممتازة ءعجبتني الاضلا...,positive
37136,ارز اللوان وين فيه ياليته طعمه زين,-1.0,RES1,ارز اللوان وين ياليته طعمه زين,negative


In [21]:
# for all test data
predict_with_key_tokens(test_df.iloc[0]["text_clean"], top_k=5)


{'label': 'positive',
 'probabilities': [0.032135214656591415,
  0.14049628376960754,
  0.8273684978485107],
 'key_words': ['الءخضر', 'ولونه', 'للعاءلات', 'العشب', 'وخلافة']}

In [22]:
results = []

for idx, row in test_df.iterrows():
    output = predict_with_key_tokens(row["text_clean"], top_k=5)

    results.append({
        "index": idx,
        "text_clean": row["text_clean"],
        "predicted_label": output["label"],
        "prob_class_0": output["probabilities"][0],
        "prob_class_1": output["probabilities"][1],
        "prob_class_2": output["probabilities"][2],
        "key_words": output["key_words"]
    })


In [23]:
results_df = pd.DataFrame(results)


In [24]:
results_df.head()


,index,text_clean,predicted_label,prob_class_0,prob_class_1,prob_class_2,key_words
0,7607,انصح انصح الفندق للعاءلات منطقة البحر المتوسط ...,positive,0.032135,0.140496,0.827368,"[الءخضر, ولونه, للعاءلات, العشب, وخلافة]"
1,42820,الذ واطعم مطبق ذقته حياتي,positive,0.001119,0.000324,0.998556,"[واطعم, الذ, ذقته, حياتي, مطبق]"
2,34907,الجو حلو ... الرياض وكنت مواعد العاءلة بطلعه ت...,positive,0.001001,0.000586,0.998413,"[حلو, مقبلات, تشكيلة, والانتظار, رحنا]"
3,36459,يتميز المطعم بوجود وجبات المنيو الرغبة بالاختي...,positive,0.000763,0.000636,0.998601,"[راءع, شي, السلطة, بوفيه, بالاختيار]"
4,23775,جيد ءكثر وجدت المطعم مرشح العديد تقارير مواقع ...,neutral,0.002502,0.988225,0.009273,"[جيد, ءكثر, جيد, مرشح, بالمميز]"


In [29]:
# evaluating the model

def eval_by_domain(df, name="domain"):
    rows = []
    for dom, part in df.groupby(name):
        ds = PandasDataset(part)
        pred = trainer.predict(ds)
        y_true = pred.label_ids
        y_pred = np.argmax(pred.predictions, axis=1)
        rows.append({
            "domain": dom,
            "n": len(part),
            "accuracy": accuracy_score(y_true, y_pred),
            "f1_macro": f1_score(y_true, y_pred, average="macro"),
        })
    return pd.DataFrame(rows).sort_values("n", ascending=False)

eval_by_domain(test_df)


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


,domain,n,accuracy,f1_macro
1,HTL,1590,0.843396,0.716536
4,RES,1113,0.957772,0.869915
5,RES1,826,0.970944,0.644831
3,PROD,400,0.807500,0.555030
6,RES2,270,0.948148,0.896297
0,AAT,208,0.966346,0.550505
2,MOV,143,0.685315,0.562727
